## Install Packages

In [34]:
from facenet_pytorch import MTCNN, InceptionResnetV1, training
import torch
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from torchvision import datasets
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import zipfile 
import torch
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
import src
from src.utils.celeba_helper import CelebADataset, CelebAClassifier, save_file_names
from imp import reload

workers = 0 if os.name == 'nt' else 2

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


# Define CelebA Dataset and Loader

In [3]:
## Load the dataset
# Path to directory with all the images
img_folder = 'data/img_align_celeba'
mapping_file = 'data/identity_CelebA.txt'

# Spatial size of training images, images are resized to this size.
image_size = 160
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size), # TODO: Experiment with fixed_image_standardization
    transforms.ToTensor()
])

# Load the dataset from file and apply transformations
celeba_dataset = CelebADataset(img_folder, mapping_file, transform)

In [4]:
## Create a dataloader
# Batch size during training
batch_size = 128
# Number of workers for the dataloader
num_workers = 0 if device.type == 'cuda' else 2
# Whether to put fetched data tensors to pinned memory
pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = DataLoader(celeba_dataset,  # type: ignore
                        batch_size=batch_size,
                        num_workers=num_workers,
                        pin_memory=pin_memory,
                        shuffle=False)

# Setup FaceNet

In [5]:
mtcnn = MTCNN(
    image_size=image_size, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, keep_all=False,
    device=device
)

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [6]:
classifier = CelebAClassifier(celeba_dataloader, detection_model=mtcnn, embedding_model=resnet)

# One-Shot Learning


## Load Dataset

In [7]:
# Select train files
file_label_mapping = celeba_dataset.get_file_label_mapping()
first_file_for_each_person_df = file_label_mapping.sort_values(by='person_id').groupby('person_id').agg(['min', 'count'])
train_files = np.sort(first_file_for_each_person_df[first_file_for_each_person_df['file_name']['count'] > 1]['file_name']['min'].values)

In [8]:
# The following takes a long time, as one image is loaded and embedded for each person. Run this once for the desired dataset. 
# Afterwards, load the data from the pickle file.

if not os.path.exists('pytorch_objects'):
  os.makedirs('pytorch_objects')

embeddings_path = 'pytorch_objects/train_embeddings_all_1img.pickle'
file_names_path = 'pytorch_objects/train_file_names_all_1img'

if not os.path.exists(embeddings_path) or not os.path.exists(file_names_path):
    train_embeddings, train_face_file_names = classifier.load_data_specific_images(files_to_load=train_files) # TODO: Replace with SubsetRandomSampler
    torch.save(train_embeddings, f'pytorch_objects/train_embeddings_all_1img.pickle')
    save_file_names(train_face_file_names, 'pytorch_objects/train_file_names_all_1img')
else:
    train_embeddings = torch.load(embeddings_path)
    train_face_file_names = []
    with open(file_names_path, 'r') as fp:
        for line in fp:
            x = line[:-1]
            # add current item to the list
            train_face_file_names.append(x)

print(f'Size of train dataset: {train_embeddings.shape}')
train_labels = celeba_dataset.get_labels_from_file_names(train_face_file_names)

Size of train dataset: torch.Size([10115, 512])
Number of people in dataset: 10115


In [9]:
# Select test files
second_file_for_each_person_df = file_label_mapping[~file_label_mapping['file_name'].isin(first_file_for_each_person_df['file_name']['min'])].sort_values(by='person_id').groupby('person_id').agg(['min', 'count'])
test_files = np.sort(second_file_for_each_person_df[second_file_for_each_person_df['file_name']['count'] >= 1]['file_name']['min'].values)

In [10]:
# The following takes a long time, as one image is loaded and embedded for each person. Run this once for the desired dataset. 
# Afterwards, load the data from the pickle file.

embeddings_path = 'pytorch_objects/test_embeddings_all_1img.pickle'
file_names_path = 'pytorch_objects/test_file_names_all_1img'

if not os.path.exists(embeddings_path) or not os.path.exists(file_names_path):
    test_embeddings, test_face_file_names = classifier.load_data_specific_images(files_to_load=test_files)
    torch.save(test_embeddings, f'pytorch_objects/test_embeddings_all_1img.pickle')
    save_file_names(test_face_file_names, 'pytorch_objects/test_file_names_all_1img')
else:
    test_embeddings = torch.load(embeddings_path)
    test_face_file_names = []
    with open(file_names_path, 'r') as fp:
        for line in fp:
            x = line[:-1]
            # add current item to the list
            test_face_file_names.append(x)
            
print(f'Size of test dataset: {test_embeddings.shape}')
test_labels = celeba_dataset.get_labels_from_file_names(test_face_file_names)

Size of test dataset: torch.Size([10117, 512])
Number of people in dataset: 10117


In [11]:
# Remove mtcnn to reduce GPU memory usage
del mtcnn

## Predicting

The following cells, each might take longer.

In [65]:
for similarity_metric in ['norm_2', 'norm_2_squared', 'cosine_similarity']:
    test_predictions, test_predictions_files = classifier.predict(test_embeddings, train_embeddings, train_face_file_names, similarity_metric)
    accuracy = accuracy_score(test_labels, test_predictions)
    print(f'Accuracy - {similarity_metric}: {np.round(accuracy, 4)}')

KeyboardInterrupt: 

In [ ]:
%%time

# fit Support Vector Classifier
from sklearn.svm import SVC
model = SVC(kernel='linear', verbose=True)
model.fit(train_embeddings, train_labels)

[LibSVM]CPU times: user 5min 57s, sys: 8min 53s, total: 14min 50s
Wall time: 14min 52s


SVC(kernel='linear', verbose=True)

In [ ]:
%%time
# train_predictions = model.predict(train_embeddings)
test_predictions = model.predict(test_embeddings)
# train_predictions = model.predict(train_embeddings)

# score_train = accuracy_score(train_labels, train_predictions)
score_test = accuracy_score(test_labels, test_predictions)
# score_train = accuracy_score(train_labels, train_predictions)

# print(f'Accuracy: train = {np.round(score_train*100, 3)}%')
print(f'Accuracy: test = {np.round(score_test*100, 3)}%')

Accuracy: test = 69.546%
CPU times: user 2h 32min 4s, sys: 11min 53s, total: 2h 43min 57s
Wall time: 2h 43min 54s


In [ ]:
%%time

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(train_embeddings, train_labels)

knn.score(test_embeddings, test_labels)

In [ ]:
#TODO: Find out which labels are misclassified and research why their embeddings are so close

# Different Loss Function

## Run default model with shuffled data

In [77]:
def get_embeddings(model, dataloader, dataset_size, batch_size):
    embeddings = torch.tensor([])
    
    for idx, batch in tqdm(enumerate(dataloader), total=int(dataset_size/batch_size)):
        imgs, batch_labels = batch
        batch_embeddings = model(imgs.to(device)).detach().cpu()
        
        if not embeddings.numel():
            embeddings = batch_embeddings
            labels = batch_labels
        else:
            embeddings = torch.cat([embeddings, batch_embeddings])
            labels = torch.cat([labels, batch_labels])
    
    return embeddings, labels

In [87]:
train_loader = DataLoader(
    celeba_dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)

val_loader = DataLoader(
    celeba_dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

test_loader = DataLoader(
    celeba_dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(test_inds)
)

In [ ]:
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [ ]:
train_embeddings, train_labels = get_embeddings(model, train_loader, len(train_files), batch_size)
test_embeddings, test_labels = get_embeddings(model, test_loader, len(test_files), batch_size)

In [79]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(train_embeddings, train_labels)
score = knn.score(test_embeddings, test_labels)

print(f'Pre-trained model: Accuracy = {score}.')

Pre-trained model: Accuracy = 0.5025165301490181.


## Fine-tune Model

In [92]:
model = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(pd.unique(file_label_mapping['person_id']))
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

In [93]:
train_inds = [int(elem[:6])-1 for elem in train_files] #convert file names to indices that start at "0"
test_inds = [int(elem[:6])-1 for elem in test_files] #convert file names to indices that start at "0"

np.random.shuffle(train_inds)
np.random.shuffle(test_inds)

In [94]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [95]:
epochs = 10

writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
model.eval()
training.pass_epoch(
    model, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    model.train()
    training.pass_epoch(
        model, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    # model.eval()
    # training.pass_epoch(
    #     model, loss_fn, val_loader,
    #     batch_metrics=metrics, show_running=True, device=device,
    #     writer=writer
    # )

writer.close()



Initial
----------
Valid |    16/16   | loss:    9.4096 | fps:  223.4194 | acc:    0.0000   

Epoch 1/10
----------
Train |    80/80   | loss:    9.3525 | fps:  169.8834 | acc:    0.0000   

Epoch 2/10
----------
Train |    80/80   | loss:    9.0378 | fps:  169.1900 | acc:    0.0001   

Epoch 3/10
----------
Train |    80/80   | loss:    8.1761 | fps:  167.0645 | acc:    0.0020   

Epoch 4/10
----------
Train |    80/80   | loss:    6.9283 | fps:  164.5172 | acc:    0.0148   

Epoch 5/10
----------
Train |    80/80   | loss:    5.6030 | fps:  166.2083 | acc:    0.0620   

Epoch 6/10
----------
Train |    80/80   | loss:    3.4869 | fps:  169.4170 | acc:    0.6521   

Epoch 7/10
----------
Train |    80/80   | loss:    3.1845 | fps:  167.6503 | acc:    0.7796   

Epoch 8/10
----------
Train |    57/80   | loss:    2.9223 | fps:  168.0489 | acc:    0.8727   

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f755fef8080>>
Traceback (most recent call last):
  File "/opt/conda/envs/one-shot-face-recognition/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/opt/conda/envs/one-shot-face-recognition/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/envs/one-shot-face-recognition/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Train |    75/80   | loss:    2.9450 | fps:  167.7559 | acc:    0.8643   

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f755fef8080>>
Traceback (most recent call last):
  File "/opt/conda/envs/one-shot-face-recognition/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/opt/conda/envs/one-shot-face-recognition/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/opt/conda/envs/one-shot-face-recognition/lib/python3.6/multiprocessing/process.py", line 134, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Train |    80/80   | loss:    2.9537 | fps:  167.1353 | acc:    0.8622   

Epoch 9/10
----------
Train |    80/80   | loss:    2.7605 | fps:  168.4049 | acc:    0.9099   

Epoch 10/10
----------
Train |    80/80   | loss:    2.5649 | fps:  166.9937 | acc:    0.9413   


In [96]:
train_embeddings, train_labels = get_embeddings(model, train_loader, len(train_files), batch_size)
test_embeddings, test_labels = get_embeddings(model, test_loader, len(test_files), batch_size)

80it [00:59,  1.35it/s]                        
80it [00:59,  1.33it/s]                        


In [109]:
x_max = torch.max(test_embeddings, 1)
x_max.indices

tensor([ 661,  751,  197,  ..., 7207, 1727, 2847])

In [110]:
test_labels

tensor([2769, 6462, 8688,  ..., 7203, 8753, 1479])

In [106]:
train_embeddings

tensor([[-1.9315e+00, -2.4994e+00,  1.1724e+00,  ..., -3.1479e+00,
         -2.4919e-01, -1.9545e+00],
        [ 4.3499e-01,  6.8356e-01, -3.6949e+00,  ..., -2.7247e+00,
         -3.6834e+00,  3.2565e+00],
        [ 5.0488e-01, -2.3267e+00, -8.6536e-01,  ...,  3.3571e+00,
          2.0080e-01,  2.8950e+00],
        ...,
        [ 4.2546e-01,  5.8745e-01,  5.8698e+00,  ..., -1.9818e+00,
         -1.2438e+00, -2.1049e+00],
        [ 3.5916e-01, -9.9086e-01,  9.8348e-01,  ..., -1.1922e-01,
         -1.8273e+00,  5.2912e-02],
        [ 1.0629e+00, -6.8456e+00,  8.8256e-01,  ...,  2.6444e+00,
         -1.2878e+00,  2.9885e-03]])

In [100]:
def min_norm_2(test_value, trained_examples):
    dists = [(test_value - anchor).norm().item() for anchor in trained_examples]
    return np.argmin(dists)

In [101]:
print(test_labels[0])
train_labels[min_norm_2(test_embeddings[0], train_embeddings)]

tensor(2769)


tensor(661)

In [97]:
knn.fit(train_embeddings, train_labels)
score = knn.score(test_embeddings, test_labels)

print(f'Fine-tuned model: Accuracy = {score}.')

Fine-tuned model: Accuracy = 0.006513372150399684.
